<a href="https://colab.research.google.com/github/SIDDHESH-8213/U-ASK-Winter-25/blob/main/U_ASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
pip install rtree

In [38]:
import numpy as np
from rtree import index
from collections import defaultdict
from math import sqrt
import pandas as pd


In [39]:
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
def load_data_from_folder(folder_path):
    dataset = []

    for root, _, files in os.walk(folder_path):
        for file in files[:1]:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)


                with open(file_path, "r") as f:
                    for line in f:
                        parts = line.strip().split()
                        obj_id = int(parts[0])
                        lat, lon = float(parts[1]), float(parts[2])
                        keyword_count = int(parts[3])


                        keywords = [parts[i] for i in range(4, 4 + keyword_count * 2, 2)]

                        dataset.append((obj_id, lat, lon, keywords))

    return dataset

dataset_folder = "/content/drive/My Drive/spatial project"

data1 = load_data_from_folder(dataset_folder)


df1 = pd.DataFrame(data1, columns=["Object ID", "Latitude", "Longitude", "Keywords"])


print(df1.head())


   Object ID   Latitude   Longitude                      Keywords
0    1500000  14.704585  121.047709                       [heart]
1    1500001 -22.919497  -43.073003                   [big, girl]
2    1500002  29.848982  -97.951838  [left, cheek, killing, back]
3    1500003  37.599676 -120.863930                      [pursue]
4    1500004  40.748709  -73.975774                   [canadians]


In [41]:
df1.size

2000000

In [42]:
spatial_idx = index.Index()


for obj_id, lat, lon, keywords in data1:
    spatial_idx.insert(obj_id, (lon, lat, lon, lat))


nearest = list(spatial_idx.nearest((-118.25, 34.0, -118.25, 34.0), 1))

print("Nearest location ID now:", nearest)


print(f"Total locations indexed: {len(data1)}")


Nearest location ID now: [1500117, 1500117, 1500117, 1500117, 1500117]
Total locations indexed: 500000


In [43]:
keyword_index = defaultdict(list)

for obj_id, lat, lon, keywords in data1:
    for keyword in keywords:
        keyword_index[keyword].append(obj_id)


print("Places that serve good burgers:", keyword_index["burgers"])


Places that serve good burgers: [1505166, 1508175, 1512668, 1515706, 1520089, 1523249, 1524363, 1541293, 1545191, 1560997, 1570645, 1574594, 1585675, 1586870, 1505166, 1508175, 1512668, 1515706, 1520089, 1523249, 1524363, 1541293, 1545191, 1560997, 1570645, 1574594, 1585675, 1586870, 1505166, 1508175, 1512668, 1515706, 1520089, 1523249, 1524363, 1541293, 1545191, 1560997, 1570645, 1574594, 1585675, 1586870, 1505166, 1508175, 1512668, 1515706, 1520089, 1523249, 1524363, 1541293, 1545191, 1560997, 1570645, 1574594, 1585675, 1586870, 1505166, 1508175, 1512668, 1515706, 1520089, 1523249, 1524363, 1541293, 1545191, 1560997, 1570645, 1574594, 1585675, 1586870]


In [44]:
def distance(lat1, lon1, lat2, lon2):
    """Calculate Euclidean distance (simplified for small areas)."""
    return sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)


In [48]:
def tkqn_query(lat, lon, positive_keywords, negative_keywords, k=3):
    nearby_ids = list(spatial_idx.nearest((lon, lat, lon, lat), k * 5))

    results = []
    for obj_id in nearby_ids:
        entry = next(d for d in data1 if d[0] == obj_id)
        obj_lat, obj_lon, obj_keywords = entry[1], entry[2], entry[3]

        print(f"DEBUG: Checking {obj_id} - Keywords: {obj_keywords}")


        if any(neg in obj_keywords for neg in negative_keywords):
            print(f" Excluded {obj_id} (Contains {negative_keywords})")
            continue


        if not any(pos in obj_keywords for pos in positive_keywords):
            print(f"Skipping {obj_id} (Doesn't contain {positive_keywords})")
            continue


        dist = distance(lat, lon, obj_lat, obj_lon)
        results.append((obj_id, dist))

    results.sort(key=lambda x: x[1])
    return results[:k]


In [52]:
top_places = tkqn_query(40.748709 , -73.975774 , ["canadians"], ["burger"], k=3)
print("Top places:", top_places)


DEBUG: Checking 1500004 - Keywords: ['canadians']
DEBUG: Checking 1500004 - Keywords: ['canadians']
DEBUG: Checking 1500004 - Keywords: ['canadians']
DEBUG: Checking 1500004 - Keywords: ['canadians']
DEBUG: Checking 1500004 - Keywords: ['canadians']
DEBUG: Checking 1593699 - Keywords: ['shit', 'write', 'couldn']
Skipping 1593699 (Doesn't contain ['canadians'])
DEBUG: Checking 1593699 - Keywords: ['shit', 'write', 'couldn']
Skipping 1593699 (Doesn't contain ['canadians'])
DEBUG: Checking 1593699 - Keywords: ['shit', 'write', 'couldn']
Skipping 1593699 (Doesn't contain ['canadians'])
DEBUG: Checking 1593699 - Keywords: ['shit', 'write', 'couldn']
Skipping 1593699 (Doesn't contain ['canadians'])
DEBUG: Checking 1593699 - Keywords: ['shit', 'write', 'couldn']
Skipping 1593699 (Doesn't contain ['canadians'])
DEBUG: Checking 1557546 - Keywords: []
Skipping 1557546 (Doesn't contain ['canadians'])
DEBUG: Checking 1557546 - Keywords: []
Skipping 1557546 (Doesn't contain ['canadians'])
DEBUG: Ch

In [51]:
def batch_queries(queries):
    results = {}


    location_groups = {}
    for query in queries:
        lat, lon, pos_keywords, neg_keywords, k = query
        key = (lat, lon)

        if key not in location_groups:
            location_groups[key] = []
        location_groups[key].append((pos_keywords, neg_keywords, k))


    for (lat, lon), query_list in location_groups.items():

        nearby_ids = list(spatial_idx.nearest((lon, lat, lon, lat), max(k for _, _, k in query_list) * 5))

        for pos_keywords, neg_keywords, k in query_list:
            query_results = []

            for obj_id in nearby_ids:
                entry = next(d for d in data1 if d[0] == obj_id)
                obj_lat, obj_lon, obj_keywords = entry[1], entry[2], entry[3]

                if any(neg in obj_keywords for neg in neg_keywords):
                    continue

                if not any(pos in obj_keywords for pos in pos_keywords):
                    continue

                dist = distance(lat, lon, obj_lat, obj_lon)
                query_results.append((obj_id, dist))

            query_results.sort(key=lambda x: x[1])
            results[(lat, lon, tuple(pos_keywords), tuple(neg_keywords), k)] = query_results[:k]

    return results


query_list = [

    (41.09032065, -85.15724857, ["wine"], ["burger"], 3),
    (38.8973979, -77.0884415, ["people"], ["burger"], 3),
    (41.32134264, -75.79291558, ["concert"], ["video"], 3),
    (-6.41991, 107.46538, ["night"], [], 3),
    (39.7424, -104.995, ["drinking"], [], 3)
]

batch_results = batch_queries(query_list)

for query, result in batch_results.items():
    print(f"\nQuery: {query}")
    print(f"Results: {result}")



Query: (41.09032065, -85.15724857, ('wine',), ('burger',), 3)
Results: []

Query: (38.8973979, -77.0884415, ('people',), ('burger',), 3)
Results: []

Query: (41.32134264, -75.79291558, ('concert',), ('video',), 3)
Results: []

Query: (-6.41991, 107.46538, ('night',), (), 3)
Results: []

Query: (39.7424, -104.995, ('drinking',), (), 3)
Results: []


In [54]:
!pip install pyspark


In [57]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, lit, array_contains
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType
import math

spark = SparkSession.builder.appName("POWER-KNN").getOrCreate()

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True),
    StructField("keyword_count", IntegerType(), True),
    StructField("keywords", StringType(), True)
])

data_path = "/content/0.txt"
df = spark.read.csv(data_path, schema=schema, sep=" ", header=False)


df = df.withColumn("keywords", split(col("keywords"), " "))


def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    return 2 * R * math.asin(math.sqrt(a))


from pyspark.sql.functions import udf
distance_udf = udf(lambda lat1, lon1, lat2, lon2: haversine(lat1, lon1, lat2, lon2), FloatType())
spark.udf.register("haversine", distance_udf)

def process_query_window(query):
    lat, lon, pos_keyword, neg_keyword, k = query


    search_radius = 10
    min_lat, max_lat = lat - search_radius, lat + search_radius
    min_lon, max_lon = lon - search_radius, lon + search_radius


    window_df = df.filter((col("latitude") >= min_lat) & (col("latitude") <= max_lat) &
                           (col("longitude") >= min_lon) & (col("longitude") <= max_lon))

    filtered_df = window_df.filter(array_contains(col("keywords"), pos_keyword) &
                                   ~array_contains(col("keywords"), neg_keyword))

    filtered_df = filtered_df.withColumn("distance", distance_udf(col("latitude"), col("longitude"), lit(lat), lit(lon)))

    results = filtered_df.orderBy("distance").limit(k)

    return results.collect()

query_list = [
    (33.8298, -118.3510, "crafts", "fabrics", 3),
    (42.3503, -71.1044, "performance", "threat", 5),
    (37.9179, 127.0628, "quadruple", "superhuman", 3),
    (-6.3190, 106.9200, "lonely", "", 3),
    (35.1199, -89.9420, "watch", "hulu", 2)
]

query_results = [process_query_window(query) for query in query_list]

flat_results = []
for i, query in enumerate(query_list):
    for row in query_results[i]:
        flat_results.append((query[0], query[1], query[2], query[3], query[4], row.id, row.latitude, row.longitude, row.distance))

flat_schema = StructType([
    StructField("query_lat", FloatType(), True),
    StructField("query_lon", FloatType(), True),
    StructField("positive_keyword", StringType(), True),
    StructField("negative_keyword", StringType(), True),
    StructField("top_k", IntegerType(), True),
    StructField("location_id", IntegerType(), True),
    StructField("location_lat", FloatType(), True),
    StructField("location_lon", FloatType(), True),
    StructField("distance", FloatType(), True)
])

results_df = spark.createDataFrame(flat_results, schema=flat_schema)

output_path = "power_results.csv"
results_df.write.csv(output_path, mode="overwrite", header=True)

print("power query saved to:", output_path)


power query saved to: power_results.csv
